# APM466 Assignment 1
### Viktar Yakauleu 1009198878

In [ ]:
!pip install numpy-financial
import numpy as np
import numpy_financial as npf
import plotly.graph_objects as go
import pandas as pd

## Import File with Data

In [ ]:
import csv

def read_csv_tuples():
    tuples_array = []

    with open("/content/APM466 A1 10 Bonds - Viktar Yakauleu.csv", newline="") as f:
        reader = csv.reader(f)
        for row_idx, row in enumerate(reader, start=1):
            if 2 <= row_idx <= 11:             # Read each row (each row is a bond), skipping the header
                row_tuples = []
                for i in range(5, 15):         # Columns 5–14 (0-indexed)
                    price = float(row[i - 1])  # Iterate over the prices for the 10 days
                    coupon_rate = float(row[1])   # Take the rate for the bond from the csv
                    maturity = float(row[3])   # Take the months to maturity for the bond from the csv

                    row_tuples.append((price, coupon_rate, maturity)) # Append each bond to the row

                tuples_array.append(row_tuples) # Append each row to the array

    return tuples_array

## Calculate Yield to Maturity

In [ ]:
bonds = read_csv_tuples()

def compute_ytm_semiannual(price, coupon_rate, maturity, face=100):
    m = 2   # Payment frequency
    periods = int(m * maturity) # Total number of periods until maturity
    coupon = coupon_rate * face / m   # Coupon payment amount

    cash_flows = [-price] + [coupon] * (periods - 1) + [coupon + face]  # Total cash flows, from formula
    irr_semi = npf.irr(cash_flows)  # Calculate the IRR/YTM using the cash flows

    if irr_semi is None or np.isnan(irr_semi):
        return np.nan

    return irr_semi * m

# Store results in an array
ytms = np.array([
    [
        compute_ytm_semiannual(price, coupon, maturity)
        for (price, coupon, maturity) in row
    ]
    for row in bonds
])

# Sort array by months until maturity
maturities = np.array([bond[0][2] for bond in bonds])

sort_idx = np.argsort(maturities)
maturities_sorted = maturities[sort_idx]
ytms_sorted = ytms[sort_idx, :]

print(ytms_sorted)

[[0.01127369 0.01127369 0.01124376 0.01125125 0.01120636 0.01122132
  0.01120636 0.0111914  0.01117644 0.01116149]
 [0.02945754 0.02944266 0.02943274 0.0294377  0.0294377  0.02943274
  0.02944762 0.02942282 0.02941786 0.02941786]
 [0.0300434  0.03005787 0.0300217  0.03002893 0.0300217  0.0300217
  0.03003616 0.03       0.03       0.02999277]
 [0.02040261 0.02040527 0.02037817 0.02038295 0.02036914 0.02037764
  0.02036436 0.02033835 0.02033782 0.02035534]
 [0.02933882 0.0293046  0.02930949 0.02929483 0.02929971 0.02977667
  0.02979152 0.02978657 0.02976183 0.02976678]
 [0.03816055 0.03812601 0.03812107 0.03810628 0.03811121 0.03809642
  0.03811614 0.03809642 0.03808164 0.03808657]
 [0.03893228 0.0388803  0.03886141 0.03887557 0.03888974 0.03887085
  0.03889919 0.0388803  0.03883311 0.03883782]
 [0.0301989  0.03022648 0.03017135 0.0301989  0.03016466 0.03018512
  0.03019496 0.03013401 0.03013204 0.03015169]
 [0.03031293 0.03034312 0.03028278 0.03031104 0.03027901 0.03029596
  0.03030539 

## Plotting the YTM Curve

In [ ]:
fig = go.Figure()

num_days = ytms_sorted.shape[1]

for day in range(num_days):
    fig.add_trace(
        go.Scatter(
            x=maturities_sorted,
            y=ytms_sorted[:, day],
            mode="lines+markers",
            name=f"Day {day + 1}",
            line=dict(width=2),
            marker=dict(size=6)
        )
    )
fig.update_layout(
    title="Yield Curves by Day",
    xaxis_title="Maturity (Months)",
    yaxis_title="Yield to Maturity",
    template="plotly_white",
    hovermode="x unified",
    legend_title_text="Trading Day"
)

fig.show()

# Calculate Spot Rates

In [ ]:
bonds_spot = read_csv_tuples()

def bootstrap_spot_curve_months(bonds_spot, face=100):
    m = 2  # Payment frequency
    spot_rates = {}  # period -> list of spot rates (one per day)

    # Sort bonds by maturity (using first day, since maturity is constant)
    bonds_spot.sort(key=lambda row: row[0][2])

    num_days = len(bonds_spot[0])

    for bond_row in bonds_spot:
        for day_idx, (price, coupon_rate, maturity_months) in enumerate(bond_row):

            periods = int(maturity_months // 6)
            coupon = coupon_rate * face / m

# Start with present value of 0 and add payments using the spot rate formula
            pv_known = 0.0
            for t in range(1, periods):
                r = spot_rates[t][day_idx]
                pv_known += coupon / (1 + r / m) ** t

            remaining = price - pv_known
            final_cf = coupon + face

            df = final_cf / remaining
            spot_rate = m * (df ** (1 / periods) - 1)

            if periods not in spot_rates:
                spot_rates[periods] = [None] * num_days

            spot_rates[periods][day_idx] = spot_rate

    return spot_rates

# Store in dictionary
spot_dict = bootstrap_spot_curve_months(bonds_spot)

print(spot_dict)

{1: [0.027333703162035317, 0.027333703162035317, 0.02692482226592041, 0.027027027027027195, 0.026413953019457814, 0.02661826981246218, 0.026413953019457814, 0.026209677419354982, 0.02600544299969787, 0.02580124974803466], 2: [0.024428320143200732, 0.024275072739168735, 0.024175976034118474, 0.024226284850176416, 0.024230858564917135, 0.024178262717109433, 0.02433301293942236, 0.024079180228445907, 0.02402964444009381, 0.0240311685639667], 3: [0.030488002162651462, 0.03062912650031091, 0.03028333679135997, 0.0303520640278645, 0.0302853855751839, 0.030284875301593406, 0.030423875864939554, 0.03007874622360962, 0.030080286676950507, 0.030011593666940506], 4: [0.023859084394837193, 0.023885079545892474, 0.02362136230390499, 0.02366758784255385, 0.023533137697633766, 0.023616873070551492, 0.02348435880738764, 0.023231648949878725, 0.023227165303951747, 0.023401301632603122], 5: [0.02438540085072649, 0.024088224915277845, 0.02413870286510278, 0.02400988210018218, 0.02405629517341712, 0.02818

# Spot Rate Graph

In [ ]:
fig = go.Figure()

# Maturities in months (sorted)
periods = sorted(spot_dict.keys())
x_vals = [period * 6 for period in periods]

# Number of days available
num_days = len(next(iter(spot_dict.values())))

# Plot the first 10 days
num_lines = min(10, num_days)

for day_idx in range(num_lines):
    y_vals = [spot_dict[period][day_idx] for period in periods]

    fig.add_trace(
        go.Scatter(
            x=x_vals,
            y=y_vals,
            mode="lines+markers",
            name=f"Day {day_idx + 1}",
            line=dict(width=2),
            marker=dict(size=6)
        )
    )

fig.update_layout(
    title="Bootstrapped Spot Curve by Day",
    xaxis_title="Maturity (Months)",
    yaxis_title="Spot Rate",
    template="plotly_white",
    hovermode="x unified",
    legend_title_text="Trading Day"
)

fig.show()

# Forward Rate Calculation

In [ ]:
def compute_1y_forward_strip(spot_rates, m=2, max_tenor=4):
    forward_rates = {}
    num_days = len(next(iter(spot_rates.values())))

    start_year = 1
    start_bond = int(m * start_year)   # = 2 for semiannual

    for tenor in range(1, max_tenor + 1):
        end_year = start_year + tenor
        end_bond = int(m * end_year)

        if end_bond not in spot_rates:
            continue

        fwd_key = (start_year, tenor)
        forward_rates[fwd_key] = [None] * num_days

        for day in range(num_days):
            s_start = spot_rates[start_bond][day]
            s_end = spot_rates[end_bond][day]

            if s_start is None or s_end is None:
                continue

            # From forward rate formula
            growth_ratio = (
                (1 + s_end) ** (m * end_year) /
                (1 + s_start) ** (m * start_year)
            )

            fwd = (growth_ratio ** (1 / (m * tenor)) - 1)
            forward_rates[fwd_key][day] = fwd

    return forward_rates

# Store in dictionary
forward_dict = compute_1y_forward_strip(spot_dict, max_tenor=4)
for k, v in forward_dict.items():
    print(f"{k[0]}y–{k[1]}y forward:", v[:5])  # first few days

1y–1y forward: [0.02329016494906977, 0.023495234842699553, 0.023067048909177768, 0.02310919559409519, 0.0228358921278915]
1y–2y forward: [0.02888611040715361, 0.0285916364250689, 0.02859999388991974, 0.028413123132782214, 0.0284705084928234]
1y–3y forward: [0.03392621778829952, 0.0342563764131929, 0.03376825661387173, 0.034016568988453955, 0.033686188415297025]
1y–4y forward: [0.034927510224302516, 0.035235540245520225, 0.03479329677461096, 0.03496615612803211, 0.034707448487291126]


# Forward Curve Graph

In [ ]:
def plot_forward_rates(forward_dict):
    """
    Plots 1y-x forward curves for each day.

    forward_dict: dict
        { (start_year, tenor) -> [fwd_rate_day1, ..., fwd_rate_dayN] }
        e.g. (1, 1) = 1y–2y, (1, 2) = 1y–3y
    """

    # Sort by tenor (1y–2y, 1y–3y, ...)
    fwd_periods = sorted(forward_dict.keys(), key=lambda x: x[1])

    # Build x-axis labels like "1–2", "1–3", ...
    x_labels = [
        f"{start}–{start + tenor - 1}"
        for start, tenor in fwd_periods
    ]

    num_days = len(next(iter(forward_dict.values())))

    fig = go.Figure()

    # One forward curve per day
    for day in range(num_days):
        y_values = [
            forward_dict[p][day] if forward_dict[p][day] is not None else None
            for p in fwd_periods
        ]

        fig.add_trace(
            go.Scatter(
                x=x_labels,
                y=y_values,
                mode="lines+markers",
                name=f"Day {day + 1}",
                connectgaps=False
            )
        )

    fig.update_layout(
        title="1-Year Forward Rate Curve (Daily)",
        xaxis_title="Forward Period (Years)",
        yaxis_title="1-Year Forward Rate",
        legend_title="Observation Day",
        template="plotly_white",
        hovermode="x unified",
    )

    fig.show()

plot_forward_rates(forward_dict)

# YTM Covariance Matrix, Eigenvalues, and Eigenvectors

In [ ]:
# Approximate dates
dates = pd.date_range("2024-01-01", periods=ytms.shape[0], freq="D")

# Select even-index entries corresponding to 1Y–5Y
maturity_indices = [1, 3, 5, 7, 9]

selected_ytms = ytms[:, maturity_indices]

# Build DataFrame
yields = pd.DataFrame(
    data=selected_ytms,
    index=dates,
    columns=["1Y", "2Y", "3Y", "4Y", "5Y"]
)

# Log returns
yield_log_returns = np.log(yields / yields.shift(1)).dropna()

# Covariance matrix
cov_yields = yield_log_returns.cov()

print(cov_yields)

# Convert covariance DataFrame to NumPy array
cov_matrix = cov_yields.values

# Eigen decomposition
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

# Sort eigenvalues and eigenvectors in descending order
idx = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

# Wrap results back into pandas objects for readability
eigenvalues = pd.Series(eigenvalues, index=cov_yields.columns, name="Eigenvalue")

eigenvectors = pd.DataFrame(
    eigenvectors,
    index=cov_yields.columns,
    columns=[f"PC{i+1}" for i in range(len(eigenvalues))]
)

print("Eigenvalues:")
print(eigenvalues)

print("\nEigenvectors:")
print(eigenvectors)

          1Y        2Y        3Y        4Y        5Y
1Y  0.175974  0.176237  0.178473  0.179094  0.179094
2Y  0.176237  0.176499  0.178739  0.179361  0.179362
3Y  0.178473  0.178739  0.181058  0.181688  0.181692
4Y  0.179094  0.179361  0.181688  0.182321  0.182325
5Y  0.179094  0.179362  0.181692  0.182325  0.182330
Eigenvalues:
1Y    8.981177e-01
2Y    6.415874e-05
3Y    4.576274e-07
4Y    1.176857e-07
5Y    8.667545e-09
Name: Eigenvalue, dtype: float64

Eigenvectors:
         PC1       PC2       PC3       PC4       PC5
1Y  0.442622 -0.553730 -0.055276  0.558747  0.426865
2Y  0.443283 -0.546750  0.115696 -0.569637 -0.408281
3Y  0.448986  0.331861 -0.258411  0.444867 -0.650843
4Y  0.450550  0.342721 -0.570783 -0.405878  0.434761
5Y  0.450557  0.408483  0.768759 -0.025912  0.196161


# Forward Covariance Matrix, Eigenvalues, and Eigenvectors

In [ ]:
def compute_log_forward_returns(forward_dict):
    log_returns = {}

    for key, rates in forward_dict.items():
        lr = []
        for j in range(len(rates) - 1):
            r_t = rates[j]
            r_tp1 = rates[j + 1]

            if r_t is None or r_tp1 is None or r_t <= 0 or r_tp1 <= 0:
                lr.append(np.nan)
            else:
                lr.append(np.log(r_tp1 / r_t))

        log_returns[key] = lr

    return log_returns

def build_forward_return_matrix(log_return_dict):
    keys = sorted(log_return_dict.keys())
    data = np.array([log_return_dict[k] for k in keys]).T

    # Drop days with missing data
    data = data[~np.isnan(data).any(axis=1)]

    return data, keys

forward_log_returns = compute_log_forward_returns(forward_dict)
forward_data, forward_keys = build_forward_return_matrix(forward_log_returns)
tenor_labels = [f"{start}Y–{tenor}Y" for start, tenor in forward_keys]

forward_cov = np.cov(forward_data, rowvar=False)

# Build DataFrame
forward_cov_df = pd.DataFrame(
    forward_cov,
    index=tenor_labels,
    columns=tenor_labels
)

print("Forward-rate covariance matrix:")
print(forward_cov_df)

# Eigen decomposition
eigvals, eigvecs = np.linalg.eigh(forward_cov)

# Sort eigenvalues and eigenvectors in descending order
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

eigvals_df = pd.Series(
    eigvals,
    index=tenor_labels,
    name="Eigenvalues"
)

print("\nEigenvalues:")
print(eigvals_df)

pc_labels = [f"PC{i+1}" for i in range(len(tenor_labels))]

# Build DataFrame
eigvecs_df = pd.DataFrame(
    eigvecs,
    index=tenor_labels,
    columns=pc_labels
)

print("\nEigenvectors:")
print(eigvecs_df)

Forward-rate covariance matrix:
          1Y–1Y     1Y–2Y     1Y–3Y     1Y–4Y
1Y–1Y  0.000162 -0.000039  0.000091  0.000085
1Y–2Y -0.000039  0.000027 -0.000023 -0.000020
1Y–3Y  0.000091 -0.000023  0.000088  0.000078
1Y–4Y  0.000085 -0.000020  0.000078  0.000070

Eigenvalues:
1Y–1Y    2.939125e-04
1Y–2Y    3.661747e-05
1Y–3Y    1.658934e-05
1Y–4Y    3.421783e-07
Name: Eigenvalues, dtype: float64

Eigenvectors:
            PC1       PC2       PC3       PC4
1Y–1Y -0.703704  0.637258 -0.307146 -0.066065
1Y–2Y  0.181444 -0.263088 -0.944057 -0.081361
1Y–3Y -0.508089 -0.570872  0.116111 -0.634405
1Y–4Y -0.462304 -0.445861 -0.030605  0.765863
